# Test

In [1]:
import numpy as np
import os
import pickle 
from itertools import combinations
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import random
from itertools import accumulate
import statistics
from statistics import mode

In [2]:
path_to_tracklet        = './tmp'
path_all_frame          = os.path.join('./data/preprocessed_data','frame_all_vec.pkl')
feat_all_vec            = os.path.join('./data/preprocessed_data','feat_all_vec.pkl')
dst_path                = './submit'

In [3]:
'''
    Một dictionary:
    keys    : scenarios id
    values  : tuple (start_cam_id, end_cam_id) dùng để truy xuất camera trong một scenarios
'''
global_cam_range = {1   :   (1,7), 
                    3   :   (14,19), 
                    9   :   (47,52), 
                    14  :   (76,81), 
                    18  :   (100,105), 
                    21  :   (118,123),
                    22  :   (124,129)}
'''
    Một dictionary:
    keys    : scenarios id
    values  : tuple (start_global_id, end_global_id) dùng để ghép ma trận so sánh
'''

global_id_scene = { 1   :   (1,8), 
                    3   :   (9,16), 
                    9   :   (17,24), 
                    14  :   (25,30), 
                    18  :   (31,38), 
                    21  :   (39,44),
                    22  :   (45,50)}

In [4]:
def feature_library(global_aug, feat_dict, global_cam_range, global_id_scene):
    global_feature_lib = {}
    weight             = {}
    for scenarios in global_cam_range.keys():
        start_cam, end_cam = global_cam_range[scenarios]
        if global_feature_lib.get(scenarios) == None:
            global_feature_lib[scenarios] = {}
        if weight.get(scenarios) == None:
            weight[scenarios] = {}    
        for cam_id in range(start_cam, end_cam + 1):
            if feat_dict.get(cam_id) == None:
                print(f'Doesn\'t find S{scenarios}')
                break
            if global_feature_lib[scenarios].get(cam_id) == None:
                global_feature_lib[scenarios][cam_id] = []
            if weight[scenarios].get(cam_id) == None:
                weight[scenarios][cam_id] = []
            start_id, end_id = global_id_scene[scenarios] 
            
            for idx in range(start_id, end_id + 1):
                sum_weight = 0
                for tracklet_id in global_aug[idx][cam_id - start_cam]:
                    global_feature_lib[scenarios][cam_id].append(feat_dict[cam_id][tracklet_id])
                    sum_weight += len(feat_dict[cam_id][tracklet_id])
                weight[scenarios][cam_id].append(sum_weight)
            global_feature_lib[scenarios][cam_id] = np.vstack(global_feature_lib[scenarios][cam_id])
    return global_feature_lib,weight

In [9]:
pickle_dict = pickle.load(open(path_all_frame, 'rb'))
feat_dict   = pickle.load(open(feat_all_vec, 'rb'))
global_aug = {}

for i in range(1,51):
    if global_aug.get(i) == None:
        global_aug[i] = []

global_aug[9] = [(1, 211, 471, 738, 45, 102, 140, 168, 242, 423, 450, 527, 676,740,791),
(225, 513, 716, 593, 148, 360, 376, 448, 651, 707, 755, 830, 871, 906),
(516, 567, 665, 695, 129, 329, 399, 401, 405, 416, 418, 427, 474, 510, 582, 630, 684, 695, 760),
(63, 127, 198, 279, 94, 115, 171, 256),
(66, 362,408, 600, 9, 66, 109, 245, 290, 451, 529, 580),
(108, 269, 511, 504, 205, 309, 356, 424, 449, 525, 563, 625, 703, 754
)]
global_aug[10] = [(3, 202, 366, 699, 244, 337,453, 479, 523, 645, 655, 790),
(1, 239,334, 698, 49, 89, 97, 152, 220, 368, 406, 742, 802, 889),
(8, 219, 316, 564, 6, 139, 132, 105, 198, 235, 268, 316,  358, 401, 439, 520, 564, 639, 690, 759),
(2,129, 209, 13, 37, 110, 129, 153, 229, 265, 305),
(75, 186,351, 402, 54, 269, 329, 498, 528, 643, 449, 320, 135),
(1, 13, 268, 491, 26, 119, 159, 300, 361, 555, 601, 617, 682, 733, 776
)]
global_aug[11] = [(2, 332, 612, 375, 35, 142, 158, 217, 219, 304, 332, 415, 432, 568, 622, 735 ),
(4, 211, 616, 717, 115, 165, 369, 431, 482, 643, 707, 803, 828, 872, 910),
(4, 569, 697, 61, 133, 181, 226, 265, 356, 423, 502, 545, 589, 638, 679, 697, 753),
(228,99,245, 210, 69, 134, 136, 296, 317),
(93, 407, 582, 515, 113, 312, 453, 486, 634, 208, 69, 67, 260),
(12, 423, 505, 671, 165, 200, 299, 394, 436, 464, 540, 571, 616, 643, 734
)]
global_aug[12] = [(4, 327, 350, 551, 65, 113, 172, 190, 327, 407, 476, 510, 551, 633, 727, 782),
(345, 521, 776, 860, 23, 144, 240,  272, 401, 480, 526, 627, 796, 860),
(1, 222, 317, 535, 29, 81, 126, 175, 217, 252, 271, 351, 458, 535, 619, 657, 680, 691, 727),
(28, 139, 104, 201, 13, 73, 139, 154, 173, 240, 282, 317,113),
(3,88, 151, 175, 52, 88, 112, 175, 244, 291, 379, 501, 577, 637),
(105, 266, 503, 655, 85, 119, 474, 603, 696, 747, 764, 565, 573 , 583
)]
global_aug[13] = [(338, 134, 267, 63, 35, 53, 60, 67, 82, 123, 159, 238, 267, 318, 357, 404, 462, 496, 601),
(236,353, 615, 713, 191, 236, 303, 466, 538, 640),
(210, 284, 541, 41, 134, 210, 253, 284, 338, 390, 433, 518, 541),
(33, 183, 119, 183, 126),
(171, 359, 442, 595, 96, 18, 153, 195, 268, 289, 366, 433),
(519, 570, 442, 556, 24, 57, 120, 292,398,418, 425, 486)]
global_aug[14] = [(647, 363, 135, 358, 56, 170, 259, 293, 335, 421, 475, 565, 620, 629, 666, 707, 774),
(246, 352,519, 609, 53, 92, 140, 269, 391, 451, 655, 801, 827, 898),
(5, 311, 563, 33, 88, 214, 232, 280, 347, 375, 455, 497, 538, 647, 682),
(102, 240, 303, 314, 17, 36, 220 , 124, 188, 303),
(91, 360, 401, 470, 116, 205, 330, 434, 551, 252, 378),
(10, 103, 270, 416, 122, 180, 190, 252, 416, 552, 566, 653, 676, 711, 721, 60 )]
global_aug[15] = [(54, 203, 341, 623, 6, 54, 114, 203, 295, 424, 463, 526, 569, 623, 669, 700, 788),
(2,254, 522, 620, 64, 159, 291, 392, 450, 511, 502, 552, 649, 691, 813, 874, 899),
(2,224, 254, 562, 46, 88, 152, 208, 325, 373, 417, 457, 546, 573, 729),
(3, 25, 275, 207, 11, 25, 89, 304, 308),
(2, 92, 147, 589, 65, 139, 169, 215, 284, 355, 415, 636, 614),
(2,  110, 409, 479, 43, 143, 152, 171, 214, 336, 557, 569, 606, 635, 677, 714, 756, 20, 54)]
global_aug[16] = [(379, 615, 128, 48, 181, 243, 294, 413, 455,  481, 508, 641, 686, 741 ),
(3, 229, 356, 505, 82, 156, 264, 353, 445, 565, 594, 660, 798, 938),
(3, 309, 538, 667, 45, 93, 96, 142, 73, 78, 107, 204, 287, 309, 379, 479, 643, 674, 757),
(137, 300, 232, 13, 179, 232,268, 298),
(1, 89, 143, 173, 83, 194, 213, 311, 385, 458, 252, 342, 404, 448, 511),
(111, 507, 554, 724, 15,  38, 51, 52, 69, 123, 131, 469, 507, 554, 672, 730, 741
)]

In [10]:
def count_group(weights, array):
    accumulate_zone = list(accumulate(map(int,weights)))
    group = []
    for i in range(len(accumulate_zone)):
      if i == 0:
        k = 0
      else:
        k = sum(group)
      group.append(np.count_nonzero(np.where(array < (accumulate_zone[i] - 1), True, False)) - k)
    return np.argmax(group)

In [11]:
def load_single_final(path_to_tracklet, path_all_frame, feat_all_vec, global_cam_range, global_id_scene, global_aug):
    pickle_dict                  = pickle.load(open(path_all_frame, 'rb'))
    feat_dict                    = pickle.load(open(feat_all_vec, 'rb'))
    print('====================== global_feature_library ======================')
    global_feature_lib, weight   = feature_library(global_aug, feat_dict, global_cam_range, global_id_scene)
    global_id_result             = {}
    print('====================== Multi_final ======================')
    for scenarios in global_cam_range.keys():
        if len(global_feature_lib[scenarios].keys()) == 0:
            print(f'Doesn\'t find S{scenarios}')
            continue
        start, end = global_cam_range[scenarios]
        start, end = int(start), int(end)
        for cam_id in range(start, end + 1):
            scenarios_feat_matrix = global_feature_lib[scenarios][cam_id]
            weights = weight[scenarios][cam_id]
            if global_id_result.get(cam_id) == None:
                global_id_result[cam_id] = {}
            start_id = global_id_scene[scenarios][0]
            for tracklet_id in pickle_dict[cam_id].keys():
                tracklet_feature    = np.array(feat_dict[cam_id][tracklet_id])
                association_matrix  = cosine_similarity(X = tracklet_feature, Y = scenarios_feat_matrix)
                global_id           = count_group(weights, np.argmax(association_matrix, axis = 1)) + int(start_id)
                global_id_result[cam_id][tracklet_id] = global_id
    return global_id_result

In [12]:
global_id_result = load_single_final(path_to_tracklet, path_all_frame, feat_all_vec, global_cam_range, global_id_scene, global_aug)

====================== global_feature_library ======================
Doesn't find S1
Doesn't find S9
Doesn't find S14
Doesn't find S18
Doesn't find S21
Doesn't find S22
====================== Multi_final ======================
Doesn't find S1
Doesn't find S9
Doesn't find S14
Doesn't find S18
Doesn't find S21
Doesn't find S22


In [ ]:
def write_output(tmp_root, dst_path, global_index):
    if not os.path.exists(os.path.dirname(dst_path)):
            os.makedirs(os.path.dirname(dst_path))

    dst_path = os.path.join(dst_path,'track1_S003.txt')    
    dst_obj = open(dst_path, 'w')
    cam_list   = sorted(os.listdir(tmp_root)) 
    for cam_file in cam_list: # Lấy cam_id
        path_tracklet   = os.path.join(tmp_root,cam_file)
        with open(path_tracklet, 'r') as fid:
            for line in fid.readlines():  
                s = [int(i) for i in line.rstrip().split()] 
                h = 1080
                w = 1920
                if s[3] < 0 or s[4] < 0 or (s[5]+s[3]) > w or (s[6]+s[4])> h:
                    continue
                s[1] = global_index[s[0]][s[1]]
                dst_obj.write('{}\n'.format(' '.join(map(str, s))))
    dst_obj.close()   

In [ ]:
dst_path                = './before_submit'

In [ ]:
write_output(path_to_tracklet, dst_path, global_id_result)

In [ ]:
global_id_result

{124: {1: 46,
  2: 45,
  3: 48,
  4: 47,
  5: 49,
  8: 50,
  11: 47,
  12: 48,
  13: 48,
  14: 48,
  15: 48,
  16: 47,
  17: 45,
  18: 45,
  20: 50,
  21: 47,
  23: 50,
  27: 47,
  28: 48,
  29: 46,
  30: 45,
  31: 45,
  32: 45,
  33: 46,
  34: 45,
  37: 50,
  38: 46,
  39: 50,
  40: 46,
  41: 49,
  43: 50,
  45: 46,
  46: 46,
  48: 48,
  49: 49,
  50: 49,
  52: 50,
  53: 48,
  54: 46,
  55: 47,
  58: 50,
  59: 48,
  60: 46,
  61: 50,
  64: 45,
  67: 50,
  68: 49,
  69: 45,
  70: 45,
  71: 49,
  72: 46,
  74: 46,
  75: 46,
  76: 49,
  77: 50,
  78: 45,
  86: 50,
  88: 50,
  92: 50,
  96: 49,
  98: 46,
  99: 46,
  102: 45,
  103: 50,
  105: 45,
  108: 50,
  109: 50,
  110: 46,
  111: 48,
  114: 49,
  115: 45,
  120: 50,
  121: 46,
  123: 50,
  124: 45,
  126: 45,
  127: 48,
  128: 45,
  129: 45,
  130: 49,
  131: 46,
  132: 50,
  133: 47,
  138: 45,
  139: 49,
  140: 48,
  141: 48,
  142: 48,
  143: 50,
  144: 50,
  145: 46,
  146: 46,
  148: 45,
  149: 45,
  150: 48,
  151: 45,
  152: 